In [1]:
from planet4.clustering import ClusteringManager
from planet4 import p4io as io
from IPython.display import display
from ipywidgets import FloatText
from pathlib import Path

In [2]:
db = io.get_current_season23_dbase()
dbname = db.dbname

In [18]:
def process_image_name(img_name):
    from pathlib import Path
    from planet4.clustering import ClusteringManager
    output_dir = Path('/Users/klay6683/Dropbox/data/planet4/inca_seasons2and3_v2')
    output_dir.mkdir(exist_ok=True)
    cm = ClusteringManager(dbname, scope='hirise',
                           output_dir=output_dir,
                           output_format='both')
    cm.cluster_image_name(img_name)
    return img_name

In [4]:
obsids = !cat /Users/klay6683/Dropbox/data/planet4/season2_3_image_names.txt

In [7]:
process_image_name(obsids[0])

'ESP_013113_0985'

In [5]:
from ipyparallel import Client
c = Client()

In [6]:
lbview = c.load_balanced_view()

In [7]:
dview = c.direct_view()

In [8]:
dview.push({'dbname': str(db.dbname)})

<AsyncResult: finished>

In [9]:
results = lbview.map_async(process_image_name, obsids)

In [10]:
from iuvs.multitools import nb_progress_display

In [11]:
nb_progress_display(results, obsids)

# Applying cut to fnotches

In [19]:
output_dir = Path('/Users/klay6683/Dropbox/data/planet4/inca_seasons2and3_v2')

In [22]:
fnotch_fnames = list(output_dir.glob('*_fnotches.hdf'))

In [180]:
def filter_for_fans(x):
    if isinstance(x, planet4.markings.Fan):
        return x

def filter_for_blotches(x):
    if isinstance(x, planet4.markings.Blotch):
        return x

In [192]:
from planet4 import markings

def process_fnotch_fname(fname):
    outpath = Path('/Users/klay6683/Dropbox/data/planet4/inca_s23_0.5cut_applied')
    outpath.mkdir(exist_ok=True)
    resman = io.ResultManager(fname)
    df = resman.fnotchdf 
    final_clusters = df.apply(markings.Fnotch.from_dataframe, axis=1).\
        apply(lambda x: x.get_marking(0.5))
    newfans = final_clusters[final_clusters.apply(filter_for_fans).notnull()]
    newblotches = final_clusters[final_clusters.apply(filter_for_blotches).notnull()]
    completefans = resman.fandf.append(newfans.apply(lambda x: x.store()),
                                       ignore_index=True)
    completeblotches = resman.blotchdf.append(newblotches.apply(lambda x: x.store()),
                                              ignore_index=True)
    completefans.to_hdf(str(outpath / resman.fanfile.name), 'df')
    completeblotches.to_hdf(str(outpath / resman.blotchfile.name), 'df')

In [193]:
for fnotch in fnotch_fnames:
    print(fnotch)
    process_fnotch_fname(fnotch)

/Users/klay6683/Dropbox/data/planet4/inca_seasons2and3_v2/ESP_011491_0985_fnotches.hdf
/Users/klay6683/Dropbox/data/planet4/inca_seasons2and3_v2/ESP_011544_0985_fnotches.hdf
/Users/klay6683/Dropbox/data/planet4/inca_seasons2and3_v2/ESP_011557_0985_fnotches.hdf
/Users/klay6683/Dropbox/data/planet4/inca_seasons2and3_v2/ESP_011623_0985_fnotches.hdf
/Users/klay6683/Dropbox/data/planet4/inca_seasons2and3_v2/ESP_011702_0985_fnotches.hdf
/Users/klay6683/Dropbox/data/planet4/inca_seasons2and3_v2/ESP_011729_0985_fnotches.hdf
/Users/klay6683/Dropbox/data/planet4/inca_seasons2and3_v2/ESP_011900_0985_fnotches.hdf
/Users/klay6683/Dropbox/data/planet4/inca_seasons2and3_v2/ESP_012256_0985_fnotches.hdf
/Users/klay6683/Dropbox/data/planet4/inca_seasons2and3_v2/ESP_012322_0985_fnotches.hdf
/Users/klay6683/Dropbox/data/planet4/inca_seasons2and3_v2/ESP_012467_0985_fnotches.hdf
/Users/klay6683/Dropbox/data/planet4/inca_seasons2and3_v2/ESP_012691_0985_fnotches.hdf
/Users/klay6683/Dropbox/data/planet4/inca_s

In [116]:
fandf = resman.fandf

In [117]:
fclusters = process_fnotch_fname(fnotch_fnames[0])

In [118]:
fandf.head()

,x,y,angle,spread,distance,arm1_x,arm1_y,arm2_x,arm2_y,n_members
0,1027.430417,66003.498891,-163.832883,40.719932,190.799632,879.396572,66014.351243,908.154802,65915.152298,58
1,850.700290,21407.542400,62.340526,66.563484,48.201664,881.127783,21424.449468,847.290043,21442.184159,43
2,1228.375000,21950.800000,-97.210326,78.351442,28.474869,1213.721703,21936.839047,1239.089801,21933.629666,4
3,783.614134,45525.612245,57.247704,49.574504,95.468890,844.311719,45564.221997,793.582345,45596.855101,49
4,729.480273,45964.978261,68.047135,62.520928,99.716577,787.613331,46008.446256,717.740384,46036.609916,46


In [128]:
isinstance(fclusters.iloc[0], planet4.markings.Blotch)

True

In [163]:
newfans = fclusters[fclusters.apply(filter_for_fans).notnull()]

In [171]:
fandf.columns

Index(['x', 'y', 'angle', 'spread', 'distance', 'arm1_x', 'arm1_y', 'arm2_x',
       'arm2_y', 'n_members'],
      dtype='object')

In [173]:
newfans.apply(lambda x: x.store()).columns

Index(['x', 'y', 'angle', 'spread', 'distance', 'arm1_x', 'arm1_y', 'arm2_x',
       'arm2_y', 'n_members'],
      dtype='object')

In [178]:
list(resman.inpath.parents)

[PosixPath('/Users/klay6683/Dropbox/data/planet4/inca_seasons2and3_v2'),
 PosixPath('/Users/klay6683/Dropbox/data/planet4'),
 PosixPath('/Users/klay6683/Dropbox/data'),
 PosixPath('/Users/klay6683/Dropbox'),
 PosixPath('/Users/klay6683'),
 PosixPath('/Users'),
 PosixPath('/')]